# Dynamic Guidance for disrupted agents

This notebook provided a quick tour on how to use the Dynamic Guidance for disrupted agents: The needed input is:

* O-D matrix of randomized trips within the examined area
* OSM map of examined area in .graphml format
* A single (or multiple users) that are in need for speed adjustment


In [175]:
import osmnx as ox
import networkx as nx
import random
import igraph as ig
import os
import time as tm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import os
from network_funcs import *

In [176]:
# Initialize globals
globals()["num_trips"] = 5
globals()["timesteps"] = 20
globals()["period"] = 60
globals()["weight"] = ["travel_car","travel_train","speed_kph","length","lanes"]
from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('rg',["g", "w","y","r"], N=256) 

In [177]:
def adjust_speed(dict_path,edges_dict_veh,G_ig,route,idx_r,time_to_complete,init_start):
    
    path_time = 0
    path_length = 0
    edges_speed = []
    edges_factor = []
    edges_length = []

    for i in route:
        path_time += G_ig.es[i]["travel_car"]
        path_length += G_ig.es[i]["length"]
        edges_speed.append(G_ig.es[i]["speed_kph"])
        edges_factor.append(G_ig.es[i]["travel_car"])
        edges_length.append(G_ig.es[i]["length"])
    edges_factor = [x/path_time for x in edges_factor]

    current_avg_velocity = sum([y*(x/path_length) for x,y in zip(edges_length,edges_speed)])
    adapted_avg_velocity = 0.001*path_length/(time_to_complete/60)
    reduction = adapted_avg_velocity/current_avg_velocity

    edge_adapted_speeds = []
    edge_adapted_times = []
    duration = init_start

    for r,i,j in zip(route,edges_speed,edges_length):
        edge_adapted_speeds.append(reduction*i)
        edge_adapted_times.append(3600*((j*0.001)/(reduction*i)))

        dict_path[r,idx_r] = (round(duration,2) ,round(duration+3600*((j*0.001)/(reduction*i)),2))
        
        if (r,step) not in edges_dict_veh.keys(): 
            edges_dict_veh[r,step] = 0

        if (step)*period <= round(duration,2) and (step+1)*period > round(duration,2):
            edges_dict_veh[r,step]+=1

        duration += 3600*((j*0.001)/(reduction*i))
    
    print(f"Desired Arrival Time : {round(step+time_to_complete,1)}")
    print(f"Unadjusted Arrival Time : {round(step+path_time/60,1)}")
    print(f"Adjusted Arrival Time : {round((sum(edge_adapted_times)+step*60)/60,1)}")
    edges_dict_veh = {x:y for x,y in edges_dict_veh.items() if y!=0}

    return dict_path,edges_dict_veh

In [178]:
def update_dict_path_adjusted(G_ig,start_truck,end_truck,routes,initial_start,orig_truck,dest_truck,dict_path,edges_dict_veh):
    time_to_finish = end_truck-step
    if time_to_finish <=0 :
        return dict_path,edges_dict_veh,routes,initial_start
    else:
        idx_t = num_trips*(start_truck+1)

        if start_truck == step:
            initial_start[idx_t] = step*60

        if initial_start[idx_t]<=(step+1)*60:
            cond = True
        else:
            cond = False
        
        if cond:
            if start_truck == step:
                route_adj = G_ig.get_shortest_paths(orig_truck,dest_truck,weights="travel_car")[0]
                routes.append(route_adj)
                initial_start[idx_t] = step*60
            else:
                route_adj =  G_ig.get_shortest_paths(routes[idx_t][0],routes[idx_t][-1],weights="travel_car")[0]
                routes[idx_t] = route_adj
                
            edge_route = []
            for i in range(len(route_adj)-1):
                edge_route.append(G_ig.get_eid(G_ig.vs[route_adj[i]], G_ig.vs[route_adj[i+1]]))    
            
            dict_path,edges_dict_veh = adjust_speed(dict_path,edges_dict_veh,G_ig,edge_route,idx_t,time_to_finish,initial_start[idx_t])
        return dict_path,edges_dict_veh,routes,initial_start

In [179]:
def generate_random_trips(G_ig,step,routes,initial_start,df):
    random.seed(step)
    size = len(routes)
    for t in range(0,num_trips):
        idx_t = size + t # Identifier of trip
        opt_weight = "travel_car"
        if np.random.rand() >=0.5:
            orig = random.choice(G_ig.vs).index
            dest = random.choice(G_ig.vs).index
        else:
            dest = random.choice(G_ig.vs).index
            orig = random.choice(G_ig.vs).index
        route = G_ig.get_shortest_paths(v=orig, to=dest, weights=opt_weight)[0]
        initial_start[idx_t] = step*period 
        routes.append(route)
    return routes,initial_start

In [180]:
# Initializing working directory and files
core_dir = os.path.join(os.getcwd(),"core\maps")
area_name = "delft_simpl_proj"
area = f"{area_name}.graphml"

# Loading OSM map from file
G = ox.load_graphml(os.path.join(core_dir,area))

# Transforming to Igraph
G,G_ig = transform_nx_to_igragh(G)
gdf_nodes,gdf_edges = ox.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

# Retaining edge info from Networkx
max_speed = get_max_speed(G_ig)

# Initialize empty train schedule
df = []

# Initialize specific truck to be adjusted
orig_truck,dest_truck,start_truck,end_truck =1,2000,3,18

In [ ]:
# Initialize Dictionaries and Lists
routes = [] # List of Networkx routes in the map (Vertex-to-Vertex)
initial_start = {}
edges_speed = {}
edges_time = {}
edges_dict_veh = {}
for step in range(timesteps):

    # Step 0 - Adjust route based on next time step
    step1 = tm.time()
    if step == 0:
        pass
    else:
        routes,initial_start,edges_dict_veh = adjust_for_next_period(G_ig,dict_path,routes,initial_start,edges_dict_veh,step)

    # Step 1 - Generate new routes
    routes,initial_start = generate_random_trips(G_ig,step,routes,initial_start,df)

    # Step 2 - Update dict_path edges and number of vehicles
    dict_path,edges_dict_veh = update_dicts(G_ig,routes,initial_start,step,edges_dict_veh)

    # Step 3 - Update gdf_edges s
    G_ig,gdf_edges = update_gdfs(G_ig,gdf_edges,edges_dict_veh,step,max_speed)

    # Step 4 - Adjust speed for specific trucks/trucks 
    if start_truck <= step:
        update_dict_path_adjusted(G_ig,start_truck,end_truck,routes,initial_start,orig_truck,dest_truck,dict_path,edges_dict_veh)
